
# COVID-19 Estimating the number of actual cases

## 1. Introduction 

We are estimating the number of infected people per day in the region. The median time from first symptom to death was estimated as 19-20 days from statistics [1, 2]. The median time from first symptom to confirmation can be estimated as a sum of incubation period and testing period [3]. We use the adjustable coefficient d_rate to match the two estimates from confirmed case and death statistics. D_rate varies depending on the country and stage of the epidemic, the height of the epidemic peak, the medical resources and the age distribution of the patients [4]. Also we assume three simplifications 
1. Assumption. The mortality rate in a given region is the same, does not change over time.
2. Assumption. Proportion of risk groups in population in a given region is the same, does not change over time.
3. Assumption. Herd immunity does not change over time.

Study Limitations

COVID-19 can proceed without symptoms (asympomatic) or in a mild form. Asympomatic cases were diagnozied for 17% of Diamond Princess passengers and 33% of Japanese citizens evacuated from Wuhan [5, 6].  


Literature
1. Wang D. et al. Clinical characteristics of 138 hospitalized patients with 2019 novel coronavirus–infected pneumonia in Wuhan, China //Jama. – 2020.
2. Linton N. M. et al. Incubation period and other epidemiological characteristics of 2019 novel coronavirus infections with right truncation: a statistical analysis of publicly available case data //Journal of Clinical Medicine. – 2020. – Т. 9. – N. 2. – С. 538.
3. https://www.worldometers.info/coronavirus/coronavirus-incubation-period/
4. Baud D. et al. Real estimates of mortality following COVID-19 infection //The Lancet Infectious Diseases. – 2020. https://doi.org/10.1016/S1473-3099(20)30195-X
5. van Doremalen N. et al. Aerosol and Surface Stability of SARS-CoV-2 as Compared with SARS-CoV-1 //New England Journal of Medicine. – 2020.
6. Mizumoto K. et al. Estimating the asymptomatic proportion of coronavirus disease 2019 (COVID-19) cases on board the Diamond Princess cruise ship, Yokohama, Japan, 2020 //Eurosurveillance. – 2020. – Т. 25. – №. 10. – С. 2000180.


In [ ]:
# run each cell by Shift+Enter
# daily_infected returns the number of infected in a given region under the assumptions 1-3. 

dayshift = 16

def estimation(timeseries, multiplier=100.0/4.0, shift=dayshift):      
    timeseries = multiplier * timeseries
    timeseries[:-shift] = timeseries[shift:]
    timeseries[-shift:] = 0
    return timeseries

import pandas as pd    
import numpy as np    
import matplotlib.pyplot as plt
from IPython.display import Image
global ax

%matplotlib inline 
plt.rcParams['figure.figsize'] = 14, 10


df = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv') 

countries = list(df['Country/Region'].unique())
def smooth(time_series,eps=0.2):    # this function smooths timeseries by three elements
    timeseries = np.zeros(len(time_series)+2)
    timeseries[1:-1] = np.array(time_series)
    timeseries[0], timeseries[-1] = timeseries[1], timeseries[-2]
    return pd.Series(eps*timeseries[:-2]+(1-2*eps)*timeseries[1:-1]+eps*timeseries[2:])

def smooth5(timeseries,eps=0.15):   # this function smooths timeseries by five elements
    timeseries = np.zeros(len(time_series)+4)
    timeseries[2:-2] = np.array(time_series)
    timeseries[0], timeseries[1], timeseries[-2], timeseries[-1] = timeseries[2], timeseries[2], timeseries[-3], timeseries[-3]
    return pd.Series(eps*timeseries[:-4]+(1/3-eps/3)*timeseries[1:-3]+(1/3-4*eps/3)*timeseries[2:-2]+(1/3-eps/3)*timeseries[3:-1]+eps*timeseries[4:])

def cum2daily(time_series):      # this function is an inverse of cummulative summation
    timeseries = np.zeros(len(time_series)+1)
    timeseries[1:] = np.array(time_series)
    timeseries[0] = timeseries[1]
    return pd.Series(timeseries[1:]-timeseries[:-1])

def show_with_dates(columns,dates, label):
    columns = pd.DataFrame(columns, columns=[label])
    columns['Date']=dates.reset_index()['Date'] 
    return columns.set_index('Date')

def show_est(data,vert_lines,d_rate=4.0,smooth_koef=0.0,shift_deaths_day=17,shift_confirm_day=5): 
    plt.figure(figsize=(14, 14), dpi=50)
    confirmed_per_day = estimation(cum2daily(smooth(data['Confirmed'],smooth_koef)),
                                   multiplier=4.0,shift=shift_confirm_day)
    ax = show_with_dates(confirmed_per_day[:-shift_confirm_day],
         data['Date'],'Infected per day, estimation by confirmed').plot.line()

    infected_per_day = estimation(cum2daily(smooth(data['Deaths'],smooth_koef)),
                                  multiplier=100.00/d_rate,shift=shift_deaths_day)
    show_with_dates(infected_per_day[:-shift_deaths_day],
         data['Date'],'Infected per day, estimation by deaths').plot.line(ax=ax)

    scale = max(confirmed_per_day)
    ax.xaxis.set_ticks(range(0,len(data['Date'])))
    ax.set_xticklabels(list(data['Date']), rotation=90)
    
    for day, y, text, color in vert_lines:
            plt.axvline(x=day,color=color)
            ax.text(day+0.5, scale * y, text,
                verticalalignment='bottom', 
                horizontalalignment='left',
                color='blue', fontsize=10)   
    plt.show()
    
    return


## 2. Reconstruction of the number of infected persons each day

In [ ]:
print('Current data set is "covid_19_clean_complete.csv" from kaggle.com by Devakumar KP')
print('data from ',df[:1]['Date'].to_string(index=False),' till ', df[-1:]['Date'].to_string(index=False))

### 2.1 Serious situations in Ukraine

In [ ]:
country = 'Ukraine'
region_df = df[df['Country/Region']==country]
 
show_est(region_df,[[9, 0.8, 'flights to and from Sanya Hainan suspended until March on 27 Jan', 'black'],
                    [30, 0.75,'Ukraine International Airlines suspended its charter service to Sanya on 4 Feb', 'black'],
                    [30, 0.7, 'Ukraine announced its first confirmed SARS-CoV-2 on 3 March', 'black'],
                    [46, 0.65, 'two more SARS-CoV-2 cases were confirmed in Ukraine 12 March', 'black'],
                    [46, 0.6, 'six more cases were confirmed in Chernivtsi Oblast on 17 March', 'black'],
                    [48, 0.55, 'the 3rd case was confirmed in Kyiv for person who had come back from Switzerland On 19 March', 'black'],
                    [59, 0.5, 'the first case of recovery from COVID-19 on 20 March', 'black'],
                    [59, 0.45, '26 new cases of COVID-19 were confimed on 22 March', 'black']
                   ],d_rate=10.0,smooth_koef=0.0,shift_deaths_day=18,shift_confirm_day=6)
#plt.yscale('log')


# https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Ukraine